# 크롤링(Crawling)
- 크롤링 Vs. 스크래핑
  - 두 용어에서 큰 차이를 두지 않고, 대표적으로 크롤링 이라는 이름으로 주로 사용
  - 구분은 가능합니다.

1. 웹 사이트에서 내용을 가져오는 방법
  - 웹 사이트에 존재하는 거의 대부분의 내용들을 가져올 수 있습니다. 
  - 텍스트, 영상, 음성, 그림, ... 
  - **주의!**
    - 마음대로 가져다가 사용해도 되는건 아닙니다.
    - 특히 **저작권**과 관련해서 문제가 발생할 수 있습니다.
    - 다운로드만 해도 불법인 경우가 있고
    - 배포가 되었을 경우에 불법인 경우가 있습니다.
      - 배포할 의도가 없었어도 배포가 되면 불법이에요
      - P2P같은 경우가 그런 경우가 많이 있습니다.
      - 변명의 여지가 없어요
    - 과도한 트랙픽을 유발할 수 있기 때문에, 관리자는 싫어합니다.
      - 일반적으로 웹 서버 관리자들은 크롤링을 허용하지 않습니다.
      - 모니터링을 통해서 차단을 합니다.

2. 오픈 API를 통해서 가져오는 방법
  - 가장 추천하는 방법 입니다.
  - 수집을 허용하기 때문에 API가 제공

## 소켓프로그래밍을 이용한 HTTP 통신
- 크롤링이란? HTTP를 이용해서 원하는 자원(파일, 그림, 영상, 텍스트, ... )을 가져오는 방법
  - 일반적으로는 웹 페이지(html)를 주로 요청
  - 그림 파일(jpg, png, ...)일 수도 있는거고
  - 여러가지 형태의 파일들을 HTTP 통신을 이용해서 가져올 수 있다. 

In [1]:
import socket

serverAddress = socket.gethostbyname('info.cern.ch')
serverPort = 80

sock = socket.socket( socket.AF_INET, socket.SOCK_STREAM )
sock.connect( (serverAddress, serverPort) )

method = 'GET' # GET 대신에 HEAD, OPTIONS로 바꿔보고 응답을 확인해봅시다 
request_header = f'{method} /index.html HTTP/1.1\r\n'
request_header += 'Host: info.cern.ch\r\n'
request_header += '\r\n'

sock.send( request_header.encode() )
response = sock.recv(1024)
print( response.decode() )

sock.close()

HTTP/1.1 200 OK
Date: Tue, 04 Jan 2022 13:47:41 GMT
Server: Apache
Last-Modified: Wed, 05 Feb 2014 16:00:31 GMT
ETag: "286-4f1aadb3105c0"
Accept-Ranges: bytes
Content-Length: 646
Connection: close
Content-Type: text/html

<html><head></head><body><header>
<title>http://info.cern.ch</title>
</header>

<h1>http://info.cern.ch - home of the first website</h1>
<p>From here you can:</p>
<ul>
<li><a href="http://info.cern.ch/hypertext/WWW/TheProject.html">Browse the first website</a></li>
<li><a href="http://line-mode.cern.ch/www/hypertext/WWW/TheProject.html">Browse the first website using the line-mode browser simulator</a></li>
<li><a href="http://home.web.cern.ch/topics/birth-web">Learn about the birth of the web</a></li>
<li><a href="http://home.web.cern.ch/about">Learn about CERN, the physics laboratory where the web was born</a></li>
</ul>
</body></html>



## 라이브러리를 이용한 HTTP 통신
- 소켓 프로그래밍을 직접 하지 않고도 편하게 HTTP 통신을 구현할 수 있습니다. 
  - urllib
  - requests

### urllib을 통한 HTTP 통신
- 파이썬 기본 패키지
- 따로 설치가 필요없이 바로 사용이 가능

In [3]:
import urllib.request

url = 'http://info.cern.ch'

request = urllib.request.Request(url)
response = urllib.request.urlopen( request )
print( response.read().decode() )

<html><head></head><body><header>
<title>http://info.cern.ch</title>
</header>

<h1>http://info.cern.ch - home of the first website</h1>
<p>From here you can:</p>
<ul>
<li><a href="http://info.cern.ch/hypertext/WWW/TheProject.html">Browse the first website</a></li>
<li><a href="http://line-mode.cern.ch/www/hypertext/WWW/TheProject.html">Browse the first website using the line-mode browser simulator</a></li>
<li><a href="http://home.web.cern.ch/topics/birth-web">Learn about the birth of the web</a></li>
<li><a href="http://home.web.cern.ch/about">Learn about CERN, the physics laboratory where the web was born</a></li>
</ul>
</body></html>



### urllib을 통한 파일 저장
- urllib은 리퀘스트 객체를 생성할 때, 헤더값을 직접 정의
  - fake_agent를 이용해서 `user-agent` 헤더의 값을 크롬 크라우저의 값과 동일하게 설정
  ```
  prompt> pip install fake_useragent
  ```

In [2]:
!pip install fake_useragent

  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-py3-none-any.whl size=13502 sha256=fa039232f75ab5a35f423e0d050dcd89b6ed86abc154a799ebc25bd31fa7fd4e
  Stored in directory: /root/.cache/pip/wheels/ed/f7/62/50ab6c9a0b5567267ab76a9daa9d06315704209b2c5d032031
Successfully built fake-useragent


In [5]:
import urllib.request
from fake_useragent import UserAgent

# 파이썬이 아닌 웹 브라우저를 통해서 요청하는 것 처럼 보이기 위해
# 헤더값을 직접 설정해줍니다.
agent = UserAgent()
header = {'User-Agent': agent.chrome}

url = 'https://image.zdnet.co.kr/2022/01/01/e601fd8d72cc33ca75cd9d41d3315684.jpg'
request = urllib.request.Request( url, headers=header )
response = urllib.request.urlopen( request )
print(response.read())

b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00\x84\x00\x06\x06\x06\x06\x07\x06\x07\x08\x08\x07\n\x0b\n\x0b\n\x0f\x0e\x0c\x0c\x0e\x0f\x16\x10\x11\x10\x11\x10\x16"\x15\x19\x15\x15\x19\x15"\x1e$\x1e\x1c\x1e$\x1e6*&&*6>424>LDDL_Z_||\xa7\x01\x06\x06\x06\x06\x07\x06\x07\x08\x08\x07\n\x0b\n\x0b\n\x0f\x0e\x0c\x0c\x0e\x0f\x16\x10\x11\x10\x11\x10\x16"\x15\x19\x15\x15\x19\x15"\x1e$\x1e\x1c\x1e$\x1e6*&&*6>424>LDDL_Z_||\xa7\xff\xc2\x00\x11\x08\x01\xaa\x02\x7f\x03\x01\x11\x00\x02\x11\x01\x03\x11\x01\xff\xc4\x00\x1d\x00\x00\x02\x02\x03\x01\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x02\x03\x04\x05\x06\x07\x08\t\xff\xda\x00\x08\x01\x01\x00\x00\x00\x00\xc2\x10\x92\x10\x90!\x00y\xefO\x12\xb8\xe2\xf0\xa2RM4\x98 \x8bBfW\xaa\xa4\x92\x10\x00\x81\x00\x02\x03~\x84\x84\x84$\x08\x01y\xefP*\xa3\x89\xc2\x02\x92q`\xd0\x87\x19D\x13\xc9\xf5h\xa4$\x08\x01\x00 \x007\xe9\x08BB@\x80G\x9d\xf5q\xad\xd7\x83\xc3\x00\xd3\x8b\x0148\xb8\xb4\x19>\xb1\x14\x90\x90\x08\x00@\x00\x83\xa0I

### urllib.request.urlretrieve를 통한 파일 저장

In [ ]:
import urllib.request
from fake_useragent import UserAgent

# UserAgent 객체 생성
agent = UserAgent()

# urllib은 urlretrieve함수를 이용해서 한 번에 파일로 저장
url = 'https://image.zdnet.co.kr/2022/01/01/e601fd8d72cc33ca75cd9d41d3315684.jpg'

# 파일을 저장할 경로
path = 'web/data/download.jpg'

# opener 객체를 생성해서 헤더를 먼저 수정해줍니다
opener = urllib.request.build_opener()
opener.addheaders = [('User-Agent', agent.chrome)]
urllib.request.install_opener(opener)

# 이제는 urlretrieve를 이용해서 다운로드 한 파일을 바로 생성할 수 있습니다
urllib.request.urlretrieve( url, path )

파일에 대한 접근이 자유롭다면 별다른 설정없이 바로 다운로드 해볼 수 있습니다. 

In [ ]:
import urllib.request

# urllib은 urlretrieve함수를 이용해서 한 번에 파일로 저장
url = 'https://unsplash.com/photos/g9P21WrK_7g/download?ixid=MnwxMjA3fDB8MXxhbGx8NHx8fHx8fDJ8fDE2NDExODI0MTk&force=true'
path = 'web/data/download2.jpg'
urllib.request.urlretrieve( url, path )

### requests을 통한 HTTP 통신
- 설치가 필요한 패키지 이지만, 콜랩에서는 미리 설치가 되어 있기 때문에 바로 사용이 가능합니다. 
  - 로컬에서 사용하는 경우에는 따로 설치를 해주면 됩니다. 

  ```
  prompt> pip install requests
  ```

- [requests 홈페이지](https://docs.python-requests.org/en/master/)

In [ ]:
import requests

url = 'http://info.cern.ch'

response = requests.get(url)
print( response.text )

<html><head></head><body><header>
<title>http://info.cern.ch</title>
</header>

<h1>http://info.cern.ch - home of the first website</h1>
<p>From here you can:</p>
<ul>
<li><a href="http://info.cern.ch/hypertext/WWW/TheProject.html">Browse the first website</a></li>
<li><a href="http://line-mode.cern.ch/www/hypertext/WWW/TheProject.html">Browse the first website using the line-mode browser simulator</a></li>
<li><a href="http://home.web.cern.ch/topics/birth-web">Learn about the birth of the web</a></li>
<li><a href="http://home.web.cern.ch/about">Learn about CERN, the physics laboratory where the web was born</a></li>
</ul>
</body></html>



### requests를 통한 파일 저장

In [ ]:
import requests
from fake_useragent import UserAgent

agent = UserAgent()
header = {'User-Agent':agent.chrome}

url = 'https://image.zdnet.co.kr/2022/01/01/e601fd8d72cc33ca75cd9d41d3315684.jpg'

response = requests.get(url, headers=header)
print( response.content )

:파일로 저장하는 경우에는 `바이너리`형태로 파일 객체를 생성하고 저장

In [ ]:
import requests
from fake_useragent import UserAgent

agent = UserAgent()
header = {'User-Agent':agent.chrome}

url = 'https://image.zdnet.co.kr/2022/01/01/e601fd8d72cc33ca75cd9d41d3315684.jpg'

response = requests.get(url, headers=header)

with open('web/data/download4.jpg', 'wb') as file:
  file.write( response.content )

# 스크래핑(Scraping)
- 웹 페이지에서 내가 원하는 내용만 가져오는 기능
  - 크롤링된 웹 페이지에서 내가 원하는 내용을 찾는 것(파싱)
  - 수업 시간에는 네이버 영화 페이지의 사용자 리뷰만 가져와보도록 합니다. 

## 네이버 영화 리뷰 페이지 크롤링
- 크롤링된 HTML내에 내가 원하는 `사용자 리뷰`가 있을겁니다. 
- 이 HTML로부터 내가 원하는 요소를 찾는겁니다(파싱)

In [ ]:
import requests

url = 'https://movie.naver.com/movie/point/af/list.naver'
response = requests.get(url)
html = response.text

## BeatifulSoup
- CSS 셀렉터를 이용해서 원하는 요소를 검색할 수 있습니다. 
- 설치

```
prompt> pip install bs4
```

In [ ]:
# BeautifulSoup 패키지를 임포트
import bs4

In [ ]:
# 수집된 HTML은 일반 텍스트 이기 때문에
# bs4 객체로 변환
review = bs4.BeautifulSoup(html)
type(review)

In [ ]:
review

## BS4를 이용한 파싱
- find, find_all, select_one, select

### find

In [ ]:
# 태그명 검색
# 일치하는 태그가 여러개인 경우에는 제일 처음 일치하는 태그의 요소를 반환
element = review.find('a')
print( type(element) )
print(element)

In [ ]:
# 일치하는 모든 요소에 대해서 리스트 형태로 반환
elements = review.find_all('a')
print(type(elements))
print(elements)

In [ ]:
# 단일 속성인 경우 
td_element = review.find('td', class_='title')
td_element

In [ ]:
# 여러개의 속성을 이용해서 검색하고자 하는 경우
# 딕셔너리 형태로 검색하고자 하는 속성을 추가해주면 됩니다. 

attrs = {
  'class':'title'
}
td_element = review.find('td', attrs=attrs)
td_element

In [ ]:
attrs = {
  'class':'title'
}
td_elements = review.find_all('td', attrs=attrs)
td_elements

In [ ]:
for string in td_elements[1].strings:
  print(string)

In [ ]:
td_elements[1].text

'\n스파이더맨: 노 웨이 홈\n\n별점 - 총 10점 중5\n\n보는 내내 영화속 일반 시민의 입장에서 생각하게 되었다 최악의 영웅놀이 잼민이다 \n\t\t\t\n\t\t\t\n\t\t\t\n\t\t\t\t\n\t\t\t\t\n\t\t\t\t\n\t\t\t\t신고\n'

In [ ]:
td_elements[1].text.split('\n')[5]

'보는 내내 영화속 일반 시민의 입장에서 생각하게 되었다 최악의 영웅놀이 잼민이다 '

In [ ]:
for child in td_elements[1].children:
  print('하위 요소들:', child)

하위 요소들: 

하위 요소들: <a class="movie color_b" href="?st=mcode&amp;sword=208077&amp;target=after">스파이더맨: 노 웨이 홈</a>
하위 요소들: 

하위 요소들: <div class="list_netizen_score">
<span class="st_off"><span class="st_on" style="width:50%">별점 - 총 10점 중</span></span><em>5</em>
</div>
하위 요소들: 

하위 요소들: <br/>
하위 요소들: 보는 내내 영화속 일반 시민의 입장에서 생각하게 되었다 최악의 영웅놀이 잼민이다 
			
			
			
				
				
				
				
하위 요소들: <a class="report" href="#" onclick="report('sddm****', '/kNca7Jrz9zPruKnmzRDoSMudVQ9ug3i/Z8k7kk6mEY=', '보는 내내 영화속 일반 시민의 입장에서 생각하게 되었다 최악의 영웅놀이 잼민이다', '17889895', 'point_after');" style="color:#8F8F8F" title="새 창">신고</a>
하위 요소들: 



완성된 코드
- 추가적으로, 영화제목과 평점도 함께 출력해 봅시다. 

In [ ]:
import requests
import bs4

url = 'https://movie.naver.com/movie/point/af/list.naver'
response = requests.get(url)
html = response.text
review = bs4.BeautifulSoup(html)

# 리뷰만 확인
search = {
  'class':'title'
}
td_elements = review.find_all('td', attrs=search)

for element in td_elements:
  print( element.text.split('\n')[5] )

### select
- select는 find_all과 동일한 기능
- select_one은 find와 동일한 기능
- CSS Selector를 이용해서 요소를 선택

In [ ]:
import requests
import bs4

url = 'https://movie.naver.com/movie/point/af/list.naver'
response = requests.get(url)
html = response.text
review = bs4.BeautifulSoup(html)

In [ ]:
review.select_one('td.title')

<td class="title">
<a class="movie color_b" href="?st=mcode&amp;sword=191600&amp;target=after">매트릭스: 리저렉션</a>
<div class="list_netizen_score">
<span class="st_off"><span class="st_on" style="width:20%">별점 - 총 10점 중</span></span><em>2</em>
</div>
<br/>매트릭스를 삼류 코미디 드라마로 만들었네..ㅎㅎ 
			
			
			
				
				
				
				<a class="report" href="#" onclick="report('pjha****', 'na4h15B/Wf0YnpiixZEh6oB/5QxJbCORvZGAmoGoPZY=', '매트릭스를 삼류 코미디 드라마로 만들었네..ㅎㅎ', '17890038', 'point_after');" style="color:#8F8F8F" title="새 창">신고</a>
</td>

In [ ]:
review.select('td.title')

[<td class="title">
 <a class="movie color_b" href="?st=mcode&amp;sword=191600&amp;target=after">매트릭스: 리저렉션</a>
 <div class="list_netizen_score">
 <span class="st_off"><span class="st_on" style="width:20%">별점 - 총 10점 중</span></span><em>2</em>
 </div>
 <br/>매트릭스를 삼류 코미디 드라마로 만들었네..ㅎㅎ 
 			
 			
 			
 				
 				
 				
 				<a class="report" href="#" onclick="report('pjha****', 'na4h15B/Wf0YnpiixZEh6oB/5QxJbCORvZGAmoGoPZY=', '매트릭스를 삼류 코미디 드라마로 만들었네..ㅎㅎ', '17890038', 'point_after');" style="color:#8F8F8F" title="새 창">신고</a>
 </td>, <td class="title">
 <a class="movie color_b" href="?st=mcode&amp;sword=85870&amp;target=after">프로메테우스</a>
 <div class="list_netizen_score">
 <span class="st_off"><span class="st_on" style="width:20%">별점 - 총 10점 중</span></span><em>2</em>
 </div>
 <br/>옆으로 뛰면 되자나... 
 			
 			
 			
 				
 				
 				
 				<a class="report" href="#" onclick="report('yjpd****', 'd873jLYpt95fLmR5LfD64NDQ2s4vszoRC2cW6GCpNNU=', '옆으로 뛰면 되자나...', '17890037', 'point_after');" style="color:

### 나머지 리뷰들도 전부 가져오려면?

In [ ]:
no = 3
url = f'https://movie.naver.com/movie/point/af/list.naver?&page={no}'

response = requests.get(url)
response.text

'\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\t\r\n\t\r\n\t\r\n\t\r\n\t\r\n\r\n\t\r\n\t\r\n\r\n\r\n\t\r\n\t\r\n\t\r\n\t\r\n\t\r\n\t\r\n\t\r\n\t\r\n\t\r\n\t\r\n\t\r\n\t\r\n\r\n\t\r\n\t\r\n\t\r\n\t\r\n\t\r\n\r\n\t\r\n\t\r\n\t\r\n\r\n\t\r\n\t\r\n\t\r\n\r\n\r\n\t\r\n\t\r\n\t\r\n\t\r\n\t\t\r\n\t\t\t\r\n\t\t\t\t\r\n\t\t\t\r\n\t\t\t\r\n\t\t\t\r\n\t\t\t\r\n\t\t\r\n\t\r\n\t\r\n\t\t\r\n\t\r\n\r\n<!DOCTYPE html>\r\n<html lang="ko">\r\n<head>\r\n<meta http-equiv="Content-Type" content="text/html; charset=UTF-8">\r\n<meta http-equiv="X-UA-Compatible" content="IE=edge">\r\n\r\n\r\n\r\n\r\n\r\n\r\n<meta property="me2:image" content="http://imgmovie.naver.com/today/naverme/naverme_profile.jpg"/>\r\n<meta property="me2:post_tag" content="네이버영화 "/>\r\n<meta property="me2:category1" content="네이버영화"/>\r\n<meta property="me2:category2" content=""/>\r\n\r\n\r\n\r\n\r\n\r\n\t\r\n\t\r\n\t\r\n\t\r\n\t\t<meta property="og:title" content="평점 : 네이버 영화"/>\r\n\t\t<meta property="og:description" content="네티즌 평점과 리뷰 정보 제

In [ ]:
url = 'https://movie.naver.com/movie/point/af/list.naver'
data = {
  'page': '1'
}

response = requests.get(url, data=data)
response.text

'\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\t\r\n\t\r\n\t\r\n\t\r\n\t\r\n\r\n\t\r\n\t\r\n\r\n\r\n\t\r\n\t\r\n\t\r\n\t\r\n\t\r\n\t\r\n\t\r\n\t\r\n\t\r\n\t\r\n\t\r\n\t\r\n\r\n\t\r\n\t\r\n\t\r\n\t\r\n\t\r\n\r\n\t\r\n\t\r\n\t\r\n\r\n\t\r\n\t\r\n\t\r\n\r\n\r\n\t\r\n\t\r\n\t\r\n\t\r\n\t\t\r\n\t\t\t\r\n\t\t\t\t\r\n\t\t\t\r\n\t\t\t\r\n\t\t\t\r\n\t\t\t\r\n\t\t\r\n\t\r\n\t\r\n\t\t\r\n\t\r\n\r\n<!DOCTYPE html>\r\n<html lang="ko">\r\n<head>\r\n<meta http-equiv="Content-Type" content="text/html; charset=UTF-8">\r\n<meta http-equiv="X-UA-Compatible" content="IE=edge">\r\n\r\n\r\n\r\n\r\n\r\n\r\n<meta property="me2:image" content="http://imgmovie.naver.com/today/naverme/naverme_profile.jpg"/>\r\n<meta property="me2:post_tag" content="네이버영화 "/>\r\n<meta property="me2:category1" content="네이버영화"/>\r\n<meta property="me2:category2" content=""/>\r\n\r\n\r\n\r\n\r\n\r\n\t\r\n\t\r\n\t\r\n\t\r\n\t\t<meta property="og:title" content="평점 : 네이버 영화"/>\r\n\t\t<meta property="og:description" content="네티즌 평점과 리뷰 정보 제

In [ ]:
search = {
  'class':'title'
}

for no in range(1, 11):
  url = f'https://movie.naver.com/movie/point/af/list.naver?&page={no}'
  response = requests.get(url)
  html = response.text
  review = bs4.BeautifulSoup(html)
  
  td_elements = review.find_all('td', attrs=search)

  for element in td_elements:
    title = element.find('a').text
    score = element.find('em').text
    r = element.text.split('\n')[5]
    print(f'영화제목: {title}')
    print(f'평점: {score}')
    print(f'리뷰: {r}')

# SELENIUM
- 기본적인 사용법

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service

options = webdriver.ChromeOptions()
options.add_experimental_option('excludeSwitches', ['enable-logging'])

# webdriver
# 크롬을 기준으로 현재 사용하고 있는 버전에 맞춰서 webdriver를 다운로드
service = Service(executable_path="c:\\IJH\\workspace\\chromedriver.exe")
browser = webdriver.Chrome(service=service, options=options)

url = 'https://www.naver.com'
browser.get( url )

# 검색어 입력 후 엔터 입력
# element = browser.find_element(By.CSS_SELECTOR, 'input#query')
# element.send_keys('검색어')
# element.send_keys('\n')

# 검색어 입력 후 마우스 클릭
# 클릭 가능한 요소라면 클릭이 가능
input = browser.find_element(By.CSS_SELECTOR, 'input#query')
button = browser.find_element(By.CSS_SELECTOR, 'button#search_btn')
input.send_keys('검색어')
button.click()

input2 = browser.find_element(By.CSS_SELECTOR, 'input#nx_query')
input2.clear()
input2.send_keys('두번째 검색어')

## 쇼핑몰 리뷰 가져오기

In [ ]:
# 두번째 실습은 쇼핑몰 후기 가져오기
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service

service = Service(executable_path="c:\\IJH\\workspace\\chromedriver.exe")
options = webdriver.ChromeOptions()
options.add_experimental_option('excludeSwitches', ['enable-logging'])
browser = webdriver.Chrome(service=service, options=options)

url = 'http://itempage3.auction.co.kr/DetailView.aspx?itemno=C253239668'
browser.get( url )

# '구매후기' 버튼클릭
review_button = browser.find_element(By.CSS_SELECTOR, 'li#tap_moving_2 a')
review_button.click()

# 구매후기 확인 
elements = browser.find_elements(By.CSS_SELECTOR, 'ul.list__review p.text')
for element in elements:
  print( element.text )

## MLP 피드 가져오기

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
import time

service = Service(executable_path="c:\\IJH\\workspace\\chromedriver.exe")
options = webdriver.ChromeOptions()
options.add_experimental_option('excludeSwitches', ['enable-logging'])
browser = webdriver.Chrome(service=service, options=options)

url = 'https://lc.multicampus.com/k-digital/#/login'
browser.get( url )

time.sleep(2) # 페이지가 로딩되는 시간을 위해 2초동안 대기
inputs = browser.find_elements(By.CSS_SELECTOR, 'div.input-row-line input')
loginButton = browser.find_element(By.CSS_SELECTOR, 'div.btn-row button.login-btn')

inputs[0].send_keys('여기가 아이디')
inputs[1].send_keys('여기가 패스워드')
loginButton.click()

# 무한 스크롤(작정하고 모든 내용을 전부 스크래핑)
# 초기 높이를 가져오고
last_height = browser.execute_script('return document.body.scrollHeight')
while True:
  # 스크롤을 아래로 내리고
  browser.execute_script('window.scrollTo(0, document.body.scrollHeight);')

  # 내용이 로딩되는 동안 잠시 기다려주고
  time.sleep(2)

  # 새로 높이를 재고
  new_height = browser.execute_script('return document.body.scrollHeight')

  # 높이가 이전과 같을 경우 더 이상 내려갈 수 없음
  if new_height == last_height: break

  # 기존의 높이를 새로운 높이로 변경
  last_height = new_height

# 로드된 내용들을 수집
articles = browser.find_elements(By.CSS_SELECTOR, 'div.feedlist span article')
for article in articles:
  for content in article.find_elements(By.CSS_SELECTOR, 'span.feedContentBlk span'):
    print( content.text )